In [1]:

from sklearn.datasets import fetch_20newsgroups
import pandas as pd

def twenty_newsgroup_to_csv():
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names)
    targets.columns=['title']

    out = pd.merge(df, targets, left_on='target', right_index=True)
    out['date'] = pd.to_datetime('now')
    out.to_csv('20_newsgroup.csv')
    
twenty_newsgroup_to_csv()

In [2]:
news=pd.read_csv('20_newsgroup.csv')
news

,Unnamed: 0,text,target,title,date
0,0,I was wondering if anyone out there could enli...,7,rec.autos,2020-02-06 14:20:17.163821
1,17,I recently posted an article asking what kind ...,7,rec.autos,2020-02-06 14:20:17.163821
2,29,\nIt depends on your priorities. A lot of peo...,7,rec.autos,2020-02-06 14:20:17.163821
3,56,an excellent automatic can be found in the sub...,7,rec.autos,2020-02-06 14:20:17.163821
4,64,: Ford and his automobile. I need information...,7,rec.autos,2020-02-06 14:20:17.163821
...,...,...,...,...,...
11309,11210,Secrecy in Clipper Chip\n\nThe serial number o...,11,sci.crypt,2020-02-06 14:20:17.163821
11310,11217,Hi !\n\nI am interested in the source of FEAL ...,11,sci.crypt,2020-02-06 14:20:17.163821
11311,11243,"The actual algorithm is classified, however, t...",11,sci.crypt,2020-02-06 14:20:17.163821
11312,11254,\n\tThis appears to be generic calling upon th...,11,sci.crypt,2020-02-06 14:20:17.163821


In [3]:
del news['Unnamed: 0']
del news['date']
del news['title']
news

,text,target
0,I was wondering if anyone out there could enli...,7
1,I recently posted an article asking what kind ...,7
2,\nIt depends on your priorities. A lot of peo...,7
3,an excellent automatic can be found in the sub...,7
4,: Ford and his automobile. I need information...,7
...,...,...
11309,Secrecy in Clipper Chip\n\nThe serial number o...,11
11310,Hi !\n\nI am interested in the source of FEAL ...,11
11311,"The actual algorithm is classified, however, t...",11
11312,\n\tThis appears to be generic calling upon th...,11


In [4]:
news.columns=['document','label']
news

,document,label
0,I was wondering if anyone out there could enli...,7
1,I recently posted an article asking what kind ...,7
2,\nIt depends on your priorities. A lot of peo...,7
3,an excellent automatic can be found in the sub...,7
4,: Ford and his automobile. I need information...,7
...,...,...
11309,Secrecy in Clipper Chip\n\nThe serial number o...,11
11310,Hi !\n\nI am interested in the source of FEAL ...,11
11311,"The actual algorithm is classified, however, t...",11
11312,\n\tThis appears to be generic calling upon th...,11


In [5]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
path='gdrive/My Drive/bert_study'

In [7]:
import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil
import warnings
import tensorflow as tf
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [8]:
!pip install keras-bert
!pip install keras-radam

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=3d8da1d4af80f62dc2fab0420e0c577d72fc4d070c9b618c16c6522b0d01e84e
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=7c7b296b237ca09708f45a2fbc6627c99beeb04d0bb324ac179e8c303d3a6334
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=e84fa095eb2d0452bb46d90071fb9d50932597fbe3907c4345f3cf9d74483bc7
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=89b312b96e3a325f0c14244eec1e4dcd9e066d67a86658ef9679b78ec1f67d88
  Stor

In [0]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam

In [0]:
SEQ_LEN = 228
BATCH_SIZE = 16#파인튜닝할땐 작게잡음
EPOCHS=2
LR=1e-5  #파인튜닝할땐 러닝레이트를 작게한다.

pretrained_path ="gdrive/My Drive/bert_study/"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = "document"
LABEL_COLUMN = "label"

In [0]:
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
          token = token.replace("_","")
          token = "##" + token
        token_dict[token] = len(token_dict)
        #단어숫자단어숫자 만들어주기위해 vocab을 가져옴

In [0]:
tokenizer = Tokenizer(token_dict)

In [0]:
#숫자로 바꿔준다 넘버링
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i])
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [0]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(news['document'], news['label'], test_size=0.3, random_state=0)

In [0]:
from IPython.display import display

train_news={"label": train_y,"document":train_x}
train_news=pd.DataFrame(train_news)

train_news.to_csv("train_news.csv", mode='w')

In [0]:
test_news={"label": test_y,"document":test_x}
test_news=pd.DataFrame(test_news)
test_news.to_csv("test_news.csv", mode='w')

In [0]:
train_news = pd.read_csv("train_news.csv")
test_news = pd.read_csv("test_news.csv")

In [18]:
train_x, train_y = load_data(train_news)
test_x, test_y = load_data(test_news)

100%|██████████| 3395/3395 [00:09<00:00, 376.26it/s]


In [0]:
layer_num = 12
model = load_trained_model_from_checkpoint( #사전학습된 버트 모형을 불러옴
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

In [20]:
model.summary() 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 228)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 228)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 228, 768), ( 81315072    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 228, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2] #우리가가저온 버트모델, 인풋이 버트는 원래3개인(mask까지)데 파인투닝할떄는 마스크 빼고 인풋 2개만 가져온다.
  dense = model.layers[-3].output #마지막 레이어를 짤라낸다.뒤에서 3번째꺼 nsp-dense 까지 자른다.그것으로 아웃풋을할것이다. 버트를가져와서 분류를 할때는 마스크를 안쓴다.!그래서 거의 항상 이렇게쓰임

#자른모델에 레이어를 달아준다. 긍정인지 부정인지 알려주는 레이어
  outputs = keras.layers.Dense(20, activation='softmax',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)



  bert_model = keras.models.Model(inputs, outputs)
  bert_model.compile(
      optimizer=RAdam(learning_rate=0.00001, weight_decay=0.0025),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])
  
  return bert_model
  

In [22]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs=2, batch_size=16, verbose = 1, validation_data=(test_x, test_y), shuffle=True)




Train on 7919 samples, validate on 3395 samples
Epoch 1/2
7919/7919 [==============================] - 320s 40ms/step - loss: 2.6459 - acc: 0.2382 - val_loss: 1.9291 - val_acc: 0.4875
Epoch 2/2
7919/7919 [==============================] - 300s 38ms/step - loss: 1.5986 - acc: 0.5878 - val_loss: 1.3038 - val_acc: 0.6406


In [0]:
def predict_convert_data(data_df):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    
    
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def predict_load_data(x): #Pandas Dataframe을 인풋으로 받는다
    data_df = x
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x = predict_convert_data(data_df)

    return data_x

In [24]:
test_set = predict_load_data(test_news)

100%|██████████| 3395/3395 [00:08<00:00, 388.40it/s]


In [0]:
preds = bert_model.predict(test_set)

In [0]:
def sentence_convert_data(data):
    global tokenizer
    indices = []
    ids, segments = tokenizer.encode(data, max_len=SEQ_LEN)
    indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def movie_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = bert_model.predict(data_x)
    predict_answer = np.round(np.ravel(predict), 0)
    predict2=bert_model.predict(data_x).argmax(axis=-1)[0]
    # if predict_answer == 0:
    #   print("this is ham")
    # elif predict_answer == 1:
    #   print("this is spam.")
    print(predict_answer)
    print('predict2: ',predict2)

In [0]:
eee='''It means that the EFF's public stance is complicated with issues irrelevant
to the encryption issue per se. There may well be people who care about
the encryption issue who don't care to associate themselves with the
network erotica issue (or may even disagree with the '''

In [37]:
movie_evaluation_predict(eee)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
predict2:  11


In [0]:
# ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
#  'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 
#  'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']